In [4]:
import calliope
#from Spanish_model.Scripts.extract_data import Extractor
model_scenario = calliope.read_netcdf('my_saved_model.nc')


In [160]:
model_scenario.build(force=True)
model_scenario.backend.verbose_strings()



[2025-08-16 17:05:10] INFO     Model: backend build starting
[2025-08-16 17:05:10] DEBUG    Math preprocessing | Loading math: ['plan']
[2025-08-16 17:05:10] INFO     Math preprocessing | added file 'plan'.
[2025-08-16 17:05:10] INFO     Math preprocessing | validated math against schema.
[2025-08-16 17:05:10] DEBUG    Optimisation model | parameters:bigM | Component not defined; using default value.
[2025-08-16 17:05:10] DEBUG    Optimisation model | parameters:bigM | Component not added; no data found in array.
[2025-08-16 17:05:10] DEBUG    Optimisation model | parameters:spores_baseline_cost | Component not defined; using default value.
[2025-08-16 17:05:10] DEBUG    Optimisation model | parameters:spores_baseline_cost | Component not added; no data found in array.
[2025-08-16 17:05:10] DEBUG    Optimisation model | parameters:spores_slack | Component not defined; using default value.
[2025-08-16 17:05:10] DEBUG    Optimisation model | parameters:spores_slack | Component not added;

In [158]:
model_scenario.solve(force=True)
model_scenario.backend.to_lp('my_saved_model.lp')


[2025-08-16 16:58:14] INFO     Optimisation model | starting model in plan mode.
Reset all parameters
Set parameter Method to value -1
Set parameter Threads to value 6
Set parameter Crossover to value 0
Set parameter BarConvTol to value 1e-05
Set parameter BarHomogeneous to value 1
Set parameter OptimalityTol to value 0.0001
Set parameter FeasibilityTol to value 0.001
Set parameter NumericFocus to value 2
Set parameter LPWarmStart to value 0
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) Ultra 5 125U, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 14 logical processors, using up to 6 threads

Non-default parameters:
FeasibilityTol  0.001
OptimalityTol  0.0001
LPWarmStart  0
BarConvTol  1e-05
BarHomogeneous  1
Crossover  0
NumericFocus  2
Threads  6

Optimize a model with 374441 rows, 221918 columns and 867632 nonzeros
Model fingerprint: 0x5e533d9a
Coefficient statistics:
  Matrix range     [1e-03, 4e+08]

In [113]:
model_scenario.backend.to_lp('my_saved_model.lp')

In [6]:
colors = model_scenario.inputs.color.to_series().to_dict()
import pandas as pd
import plotly.express as px
import plotly.io as pio
pio.templates.default = "plotly_white"

AttributeError: 'Dataset' object has no attribute 'color'

In [144]:
df_capacity = (
    model_scenario.results.flow_cap
    .where(model_scenario.results.techs != "demand_tech")
    .sel(carriers="electricity")
    .to_series()
    .where(lambda x: x != 0)
    .dropna()
    .to_frame("Flow capacity (MW)")
    .reset_index()
)

# Aggregate by technology

# Aggregate flow capacity per technology
df_tech_capacity = (
    df_capacity
    .groupby("techs", as_index=False)
    .agg({"Flow capacity (MW)": "sum"})
    .sort_values("Flow capacity (MW)", ascending=False)  # Optional: sort
)

# Plot with vertical bars (default)
fig = px.bar(
    df_tech_capacity,
    x="techs",  # techs on x-axis
    y="Flow capacity (MW)",  # capacity on y-axis
    title="Total Electricity Flow Capacity per Technology",
    color="techs",
    color_discrete_map=colors,
)

# Optional: white background
import plotly.io as pio
pio.templates.default = "plotly_white"

fig.show()

df_tech_capacity



,techs,Flow capacity (MW)
3,ESP_2_ESP_3_ac_ohl_transmission,38200.000
31,existing_chp,24704.730
40,existing_wind_onshore,22777.144
10,ESP_3_ESP_7_ac_ohl_transmission,20400.000
23,ESP_7_ESP_8_ac_ohl_transmission,17400.000
27,ESP_9_ESP_10_ac_ohl_transmission,16200.000
1,ESP_1_ESP_2_ac_ohl_transmission,14400.000
8,ESP_3_ESP_4_ac_ohl_transmission,14200.000
16,ESP_5_ESP_6_techs_ac_ohl_transmission,14000.000
25,ESP_8_ESP_9_ac_ohl_transmission,12200.000


In [122]:
df_capacity = (
    model_scenario.results.flow_in.where(model_scenario.results.techs != "demand_tech")
    .sel(carriers="electricity")
    .to_series()
    .where(lambda x: x != 0)
    .dropna()
    .to_frame("Flow capacity (MW)")
    .reset_index()
)

print(df_capacity.head(40))

fig = px.bar(
    df_capacity,
    x="nodes",
    y="Flow capacity (MW)",
    color="techs",
    color_discrete_map=colors,
)
fig.show()

Empty DataFrame
Columns: [nodes, techs, timesteps, Flow capacity (MW)]
Index: []


In [116]:
model_scenario.to_csv('output',allow_overwrite=True)

In [123]:
df_capacity = (
    model_scenario.results.flow_out.where(model_scenario.results.techs != "demand_tech")
    .sel(carriers="electricity")
    .to_series()
    .where(lambda x: x != 0)
    .dropna()
    .to_frame("Flow production (MWh)")
    .reset_index()
)
df_capacity

,nodes,techs,timesteps,Flow production (MWh)
0,ESP_1,existing_biogas_to_electricity_supply,2018-01-01 00:00:00,23.211
1,ESP_1,existing_biogas_to_electricity_supply,2018-01-01 03:00:00,23.211
2,ESP_1,existing_biogas_to_electricity_supply,2018-01-01 06:00:00,23.211
3,ESP_1,existing_biogas_to_electricity_supply,2018-01-01 09:00:00,23.211
4,ESP_1,existing_biogas_to_electricity_supply,2018-01-01 12:00:00,23.211
...,...,...,...,...
1755,ESP_9,existing_biomass,2018-01-10 09:00:00,273.159
1756,ESP_9,existing_biomass,2018-01-10 12:00:00,273.159
1757,ESP_9,existing_biomass,2018-01-10 15:00:00,273.159
1758,ESP_9,existing_biomass,2018-01-10 18:00:00,273.159


In [32]:
df_capacity = (
    model_scenario.results.flow_out.where(model_scenario.results.techs != "demand_tech")
    .sel(carriers="electricity")
    .to_series()
    .where(lambda x: x != 0)
    .dropna()
    .to_frame("Flow production (MWh)")
    .reset_index()
)

#print(df_capacity.head(40))
df_capacity = df_capacity.drop(columns=['timesteps'])

# Ensure 'flow' is numeric (optional but safe)


# Group by 'technology' and sum the 'flow'
df_capacity = df_capacity.groupby('techs', as_index=False)['Flow production (MWh)'].sum()
total= df_capacity['Flow production (MWh)'].sum()
fig.show()
df_capacity
total






283147677.4038518

In [ ]:
# Get the total systems' production

In [124]:
df_capacity = (
    model_scenario.results.flow_out.where(model_scenario.results.techs != "demand_tech")
    .sel(carriers="electricity")
    .to_series()
    .where(lambda x: x != 0)
    .dropna()
    .to_frame("Flow production (MWh)")
    .reset_index()
)

#print(df_capacity.head(40))
df_capacity = df_capacity.drop(columns=['timesteps'])
production=df_capacity['Flow production (MWh)'].sum()
production

64576.08000000001

In [125]:
total_demand = model_scenario.results.flow_in.sel(techs='demand_tech', carriers='electricity').sum().item()
print(f"Total demand from flow_in to demand_tech: {total_demand} MWh")

Total demand from flow_in to demand_tech: 6888982.668291358 MWh


In [126]:
df_capacity = (
    model_scenario.results.flow_in
    .where(
        (model_scenario.results.techs != "demand_tech") &
        (model_scenario.inputs.base_tech != "transmission")
    )
    .sel(carriers="electricity")
    .to_series()
    .where(lambda x: x != 0)
    .dropna()
    .to_frame("Flow production (MWh)")
    .reset_index()
)


print(df_capacity.head(40))

fig = px.bar(
    df_capacity,
    x="nodes",
    y="Flow production (MWh)",
    color="techs",
    color_discrete_map=colors,
)
fig.show()

Empty DataFrame
Columns: [nodes, techs, timesteps, Flow production (MWh)]
Index: []


In [50]:
df_electricity = (
    (model_scenario.results.flow_out.fillna(0) - model_scenario.results.flow_in.fillna(0))
    .sel(carriers="electricity")
    .sum("nodes")
    .to_series()
    .where(lambda x: x != 0)
    .dropna()
    .to_frame("Flow in/out (kWh)")
    .reset_index()
)
df_electricity_demand = df_electricity[df_electricity.techs == "demand_tech"]
df_electricity_other = df_electricity[df_electricity.techs != "demand_tech"]

print(df_electricity.head(15))

fig1 = px.bar(
    df_electricity_other,
    x="timesteps",
    y="Flow in/out (kWh)",
    color="techs",
    color_discrete_map=colors,
)
fig1.add_scatter(
    x=df_electricity_demand.timesteps,
    y=-1 * df_electricity_demand["Flow in/out (kWh)"],
    marker_color="black",
    name="demand",
)



                                techs           timesteps  Flow in/out (kWh)
0   ESP_10_ESP_11_ac_ohl_transmission 2018-01-01 00:00:00          -0.106556
1   ESP_10_ESP_11_ac_ohl_transmission 2018-01-01 03:00:00          -0.106558
2   ESP_10_ESP_11_ac_ohl_transmission 2018-01-01 06:00:00          -0.106570
3   ESP_10_ESP_11_ac_ohl_transmission 2018-01-01 09:00:00          -0.106562
4   ESP_10_ESP_11_ac_ohl_transmission 2018-01-01 12:00:00          -0.106551
5   ESP_10_ESP_11_ac_ohl_transmission 2018-01-01 15:00:00          -0.106564
6   ESP_10_ESP_11_ac_ohl_transmission 2018-01-01 18:00:00          -0.106557
7   ESP_10_ESP_11_ac_ohl_transmission 2018-01-01 21:00:00          -0.106559
8   ESP_10_ESP_11_ac_ohl_transmission 2018-01-02 00:00:00          -0.106557
9   ESP_10_ESP_11_ac_ohl_transmission 2018-01-02 03:00:00          -0.106560
10  ESP_10_ESP_11_ac_ohl_transmission 2018-01-02 06:00:00          -0.106561
11  ESP_10_ESP_11_ac_ohl_transmission 2018-01-02 09:00:00          -0.106549

KeyError: ''

In [79]:
df_electricity = (
    (model_scenario.results.flow_out.fillna(0) - model_scenario.results.flow_in.fillna(0))
    .sel(carriers="electricity")
    .to_series()
    .where(lambda x: x != 0)
    .dropna()
    .to_frame("Flow in/out (kWh)")
    .reset_index()
)
df_electricity_demand = df_electricity[df_electricity.techs == "demand_tech"]
df_electricity_other = df_electricity[df_electricity.techs != "demand_tech"]

print(df_electricity.head())

node_order = df_electricity.nodes.unique()

fig = px.bar(
    df_electricity_other,
    x="timesteps",
    y="Flow in/out (kWh)",
    facet_row="nodes",
    color="techs",
    category_orders={"nodes": node_order},
    height=1000,
    color_discrete_map=colors,
)

showlegend = True
# we reverse the node order (`[::-1]`) because the rows are numbered from bottom to top.
for idx, node in enumerate(node_order[::-1]):
    demand_ = df_electricity_demand.loc[
        df_electricity_demand.nodes == node, "Flow in/out (kWh)"
    ]
    if not demand_.empty:
        fig.add_scatter(
            x=df_electricity_demand.loc[
                df_electricity_demand.nodes == node, "timesteps"
            ],
            y=-1 * demand_,
            row=idx + 1,
            col="all",
            marker_color="black",
            name="Demand",
            legendgroup="demand",
            showlegend=showlegend,
        )
        showlegend = False
fig.update_yaxes(matches=None)
fig.show()

   nodes                            techs           timesteps  \
0  ESP_1  ESP_1_ESP_2_ac_ohl_transmission 2018-01-01 00:00:00   
1  ESP_1  ESP_1_ESP_2_ac_ohl_transmission 2018-01-01 03:00:00   
2  ESP_1  ESP_1_ESP_2_ac_ohl_transmission 2018-01-01 06:00:00   
3  ESP_1  ESP_1_ESP_2_ac_ohl_transmission 2018-01-01 09:00:00   
4  ESP_1  ESP_1_ESP_2_ac_ohl_transmission 2018-01-01 12:00:00   

   Flow in/out (kWh)  
0      -16461.961398  
1      -16482.995537  
2      -16481.727471  
3      -16461.590075  
4      -16484.518471  


In [127]:
df_storage_cap = (
    model_scenario.results.storage_cap.to_series()
    .where(lambda x: x != 0)
    .dropna()
    .to_frame("Storage capacity (kWh)")
    .reset_index()
)

print(df_capacity.head())

fig = px.bar(
    df_storage_cap,
    x="nodes",
    y="Storage capacity (kWh)",
    color="techs",
    color_discrete_map=colors,
)
fig.show()

Empty DataFrame
Columns: [nodes, techs, timesteps, Flow production (MWh)]
Index: []


In [130]:
df_coords = model_scenario.inputs[["latitude", "longitude"]].to_dataframe().reset_index()
df_capacity = (
    model_scenario.results.flow_cap.where(model_scenario.inputs.base_tech == "transmission")
    .to_series().to_frame("Flow capacity (kW)")
    .reset_index()
)
df_capacity_coords = pd.merge(df_coords, df_capacity, left_on="nodes", right_on="nodes")

fig = px.line_map(
    df_capacity_coords,
    lat="latitude",
    lon="longitude",
    color="techs",
    hover_data="Flow capacity (kW)",
    zoom=3,
    height=300,
)
fig.update_layout(
    mapbox_style="open-street-map",
    mapbox_zoom=4,
    mapbox_center_lat=df_coords.latitude.mean(),
    mapbox_center_lon=df_coords.longitude.mean(),
    margin={"r": 0, "t": 0, "l": 0, "b": 0},
)
df_capacity

,nodes,techs,carriers,Flow capacity (kW)
0,ESP_1,ESP_10_ESP_11_ac_ohl_transmission,coal,NaN
1,ESP_1,ESP_10_ESP_11_ac_ohl_transmission,electricity,NaN
2,ESP_1,ESP_10_ESP_11_ac_ohl_transmission,natural_gas,NaN
3,ESP_1,ESP_10_ESP_11_ac_ohl_transmission,oil,NaN
4,ESP_1,ESP_10_ESP_11_ac_ohl_transmission,uranium,NaN
...,...,...,...,...
13225,Trade_Markets,wind_onshore_monopoly_PT,coal,NaN
13226,Trade_Markets,wind_onshore_monopoly_PT,electricity,NaN
13227,Trade_Markets,wind_onshore_monopoly_PT,natural_gas,NaN
13228,Trade_Markets,wind_onshore_monopoly_PT,oil,NaN


In [145]:
model_scenario.to_csv('outputt', allow_overwrite=True)

In [84]:
df = pd.read_csv(r"C:\Users\1496051\PycharmProjects\Calliope_Spain\Spanish_model\model\outputt\inputs_sink_equals.csv")

df=df['sink_equals'].sum()
df

-263000000.00000006

In [124]:
path= r'C:\Users\1496051\PycharmProjects\Calliope_Spain\Spanish_model\model\output3\results_flow_out.csv'
df=pd.read_csv(path)
sum = df['flow_out'].sum()
sum

7910527.589384967

In [39]:
model_scenario.inputs



<xarray.Dataset> Size: 202MB
Dimensions:                       (costs: 3, techs: 147, nodes: 18,
                                   carriers: 5, time: 8760, timesteps: 752)
Coordinates:
  * techs                         (techs) object 1kB 'ESP_10_ESP_11_ac_ohl_tr...
  * carriers                      (carriers) <U11 220B 'coal' ... 'uranium'
  * nodes                         (nodes) object 144B 'ESP_1' ... 'Trade_Mark...
  * timesteps                     (timesteps) datetime64[ns] 6kB 2018-01-01 ....
  * costs                         (costs) object 24B 'co2' 'monetary' 'vul'
  * time                          (time) object 70kB '2018-01-01 00:00:00' .....
Data variables: (12/41)
    co2_limit                     (costs) float64 24B 2.06e+10 nan nan
    objective_cost_weights        (costs) float64 24B 0.0 1.0 0.0
    base_tech                     (techs) object 1kB 'transmission' ... 'supply'
    carrier_out                   (nodes, techs, carriers) float64 106kB nan ...
    cost_flow_out                 (techs, costs) float64 4kB nan nan ... nan nan
    cost_om_annual                (techs, costs) float64 4kB nan nan ... nan
    ...                            ...
    definition_matrix             (nodes, techs, carriers) bool 13kB False .....
    color                         (techs) <U7 4kB '#19122b' ... '#d486af'
    distance                      (techs) float64 1kB 248.9 185.4 ... nan nan
    source_use_max                (techs, timesteps, nodes) float64 16MB nan ...
    timestep_resolution           (timesteps) float64 6kB 3.0 3.0 ... 3.0 3.0
    timestep_weights              (timesteps) float64 6kB 1.0 1.0 ... 1.0 1.0
Attributes: (12/13)
    termination_condition:         optimal
    calliope_version_defined:      0.7.0.dev6
    calliope_version_initialised:  0.7.0.dev6
    applied_overrides:             monetary_testing
    scenario:                      cost_optimization
    defaults:                      {'bigM': 1000000000.0, 'objective_cost_wei...
    ...                            ...
    name:                          Calliope Spain
    timestamp_model_creation:      1755183015.734854
    timestamp_build_start:         1755183080.228691
    timestamp_build_complete:      1755183191.42043
    timestamp_solve_start:         1755183207.881493
    timestamp_solve_complete:      1755183257.985729

In [9]:
import pandas as pd
df = pd.read_csv(r'C:\Users\1496051\PycharmProjects\calliope\src\calliope\example_models\spanish_model\output_dd\results_flow_out.csv')
df["timesteps"] = pd.to_datetime(df["timesteps"])

# Group by nodes, techs, carriers and sum flow_out
df_agg = df.groupby(["nodes", "techs", "carriers"], as_index=False)["flow_out"].sum()

df_agg


,nodes,techs,carriers,flow_out
0,ESP_1,ESP_1_ESP_2_ac_ohl_transmission,electricity,0.451464
1,ESP_1,ESP_1_PRT_1_ac_ohl_transmission,electricity,0.080571
2,ESP_1,Trade_Markets_ESP_1_coal_transmission,coal,0.000138
3,ESP_1,Trade_Markets_ESP_1_natural_gas_transmission,natural_gas,38470.971274
4,ESP_1,battery_CN,electricity,0.022036
...,...,...,...,...
835,Trade_Markets,oil_NGA,oil,0.000000
836,Trade_Markets,oil_SA,oil,0.000000
837,Trade_Markets,oil_USA,oil,0.000000
838,Trade_Markets,uranium_UK,uranium,0.000000
